# Análisis Exploratorio de Datos: Estudio del Dengue en Sonora utilizando SQLite3

In [24]:
# Importar las librarías SQLite3 y Pandas.
import sqlite3
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [25]:
# Nombre de tu archivo de base de datos
db_filename = './database/enfermedades_db.db'

In [26]:
# Obtén la ruta completa del archivo de la base de datos
db_path = os.path.join(os.getcwd(), db_filename)

In [27]:
# Crea una conección a la base de datos SQLite.
connection = sqlite3.connect(db_path)

In [28]:
# Crea un objeto Cursor para ejecutar instrucciones en SQL.
cursor = connection.cursor()

In [29]:
# Consulta SQL para obtener la lista de tablas
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
tables = cursor.fetchall()

# Imprimir la lista de tablas
print("Tablas en la base de datos:")
for table in tables:
    print(table[0])


Tablas en la base de datos:
municipio
dictamen
entidad
estatus_caso
institucion
resultado_pcr
sexo
si_no
tipo_paciente
registro


## 1. ¿Cuales son los municipios de sonora con más casos de dengue?


In [30]:
# Comenzamos investigando qué periodos de tiempo comprende la base de datos
query_periodo_inicial = """
SELECT *
FROM registro
ORDER BY FECHA_ACTUALIZACION ASC
LIMIT 1;
"""
# Lectura de datos desde SQLite
df_inicio = pd.read_sql_query(query_periodo_inicial, connection)

# Imprimir el DataFrame resultante
df_inicio

,Unnamed: 0,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,3861,2022-01-06,925362,1,44,5,28,2,2,5,...,2,2,2,2,5.0,1,5,3,5,28


In [31]:
# Comenzamos investigando qué periodos de tiempo comprende la base de datos
query_periodo_final = """
SELECT *
FROM registro
ORDER BY FECHA_ACTUALIZACION DESC
LIMIT 1;
"""
# Lectura de datos desde SQLite
df_final = pd.read_sql_query(query_periodo_final, connection)

# Imprimir el DataFrame resultante
df_final

,Unnamed: 0,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,56274,2022-12-12,941045,2,43,24,23,2,2,24,...,2,2,2,2,5.0,1,5,3,24,23


Podemos ver que la base de datos tiene registros desde el 06 de enero de 2022 hasta el 12 de diciembre del 2022. Procedemos entonces a investigar la pregunta central.

In [32]:
query_municipios = """
    SELECT
        MUNICIPIO_DESC,
        COUNT(*) AS TOTAL_CASOS
    FROM
        (
            SELECT
                municipio.MUNICIPIO AS MUNICIPIO_DESC
            FROM
                registro
                INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
                INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_CASO = 2
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        MUNICIPIO_DESC;
"""

# Lectura de datos desde SQLite
df_municipios = pd.read_sql_query(query_municipios, connection)

# Imprimir el DataFrame resultante
df_municipios

,MUNICIPIO_DESC,TOTAL_CASOS
0,AGUA PRIETA,4
1,ALAMOS,3544
2,ALTAR,6
3,ARIVECHI,9
4,BENITO JUÁREZ,256
5,BÁCUM,277
6,CABORCA,235
7,CAJEME,3812
8,CANANEA,18
9,EMPALME,176


In [33]:
df_municipios = df_municipios.sort_values(by='TOTAL_CASOS')

# Ajustar el tamaño de la figura y habilitar el scroll
fig = px.bar(df_municipios, x='TOTAL_CASOS', y='MUNICIPIO_DESC',
             labels={'TOTAL_CASOS': 'Total de Casos', 'MUNICIPIO_DESC': 'Municipio'},
             title='Total de Casos de Dengue por Municipio',
             width=1000, height=800)  # Ajusta el ancho y alto según tus necesidades

# Habilitar el scroll en el eje y (para ver todas las etiquetas)
fig.update_layout(yaxis=dict(showline=True, showgrid=False))

# Mostrar la gráfica interactiva
fig.show()

En el gráfico anterior, se destaca claramente que Navojoa registra la mayor cantidad de casos de dengue confirmados. La magnitud de la diferencia con respecto a otros municipios es notoria, sugiriendo que podría existir un factor específico que contribuya a esta disparidad. Esta discrepancia sustancial en la cantidad de casos sugiere que Navojoa podría ser una región más propensa a un brote de dengue.

### 1.1 ¿Navojoa será el municipio con mayor casos de dengue confirmados historicamente?

In [34]:
query_municipios_hist = """
    SELECT
        MUNICIPIO_DESC,
        FECHA_SIGN_SINTOMAS,
        COUNT(*) AS TOTAL_CASOS
    FROM
        (
            SELECT
                municipio.MUNICIPIO AS MUNICIPIO_DESC,
                FECHA_SIGN_SINTOMAS
            FROM
                registro
                INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_CASO = 2
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        MUNICIPIO_DESC, FECHA_SIGN_SINTOMAS;

"""

# Lectura de datos desde SQLite
df_municipios_hist = pd.read_sql_query(query_municipios_hist, connection)

#Se cambian a objetos datetime los formatos mixtos
date1 = pd.to_datetime(df_municipios_hist['FECHA_SIGN_SINTOMAS'], errors='coerce', format='%Y-%m-%d')
date2 = pd.to_datetime(df_municipios_hist['FECHA_SIGN_SINTOMAS'], errors='coerce', format='%d/%m/%Y')
df_municipios_hist['FECHA_SIGN_SINTOMAS'] = date1.fillna(date2)
#Se organizan las fechas
df_municipios_hist = df_municipios_hist.sort_values(by='FECHA_SIGN_SINTOMAS')

# Imprimir el DataFrame resultante
df_municipios_hist

,MUNICIPIO_DESC,FECHA_SIGN_SINTOMAS,TOTAL_CASOS
23,ALAMOS,2021-02-08,1
156,BÁCUM,2021-09-09,1
474,HERMOSILLO,2021-09-15,1
157,BÁCUM,2021-09-18,1
158,BÁCUM,2021-09-19,2
...,...,...,...
232,CABORCA,2022-11-28,2
741,NAVOJOA,2022-11-28,2
742,NAVOJOA,2022-11-29,2
558,HERMOSILLO,2022-12-01,2


In [35]:
# Crear una nueva columna 'MES_ANO' que contenga la combinación de mes y año
df_municipios_hist['MES_ANO'] = df_municipios_hist['FECHA_SIGN_SINTOMAS'].dt.to_period('M')

# Convertir 'MES_ANO' a string
df_municipios_hist['MES_ANO'] = df_municipios_hist['MES_ANO'].astype(str)

# Agrupar por 'MES_ANO' y 'MUNICIPIO_DESC', sumando los casos
df_municipios_hist_grouped = df_municipios_hist.groupby(['MES_ANO', 'MUNICIPIO_DESC']).agg({'TOTAL_CASOS': 'sum'}).reset_index()

# Crea la gráfica interactiva con Plotly Express
fig = px.line(df_municipios_hist_grouped, x='MES_ANO', y='TOTAL_CASOS', color='MUNICIPIO_DESC',
              labels={'TOTAL_CASOS': 'Total de Casos', 'MES_ANO': 'Mes y Año', 'MUNICIPIO_DESC': 'Municipio'},
              title='Comportamiento Histórico de total de casos de dengue por Municipio (Agrupado por Mes-Año)')

# Muestra la gráfica interactiva
fig.show()

En la visualización, destaca la notable cantidad de casos de dengue en Navojoa, alcanzando un pico pronunciado en septiembre. Este aumento podría asociarse con las fuertes lluvias experimentadas durante ese mes. La correlación entre las precipitaciones y los casos de dengue resalta la importancia de factores climáticos en la propagación de la enfermedad. La identificación de estos patrones puede ser clave para implementar medidas preventivas efectivas en el futuro.

## 2. ¿Cómo se distribuye la cantidad de casos confirmados por dengue? 

Comenzamos creando un dataframe con la información de los registros que nos interesan, así generamos una tabla con las variables:

- ID_REGISTRO
- SEXO 
- EDAD_ANOS 
- MUNICIPIO_RES
- DICTAMEN
- RESULTADO_PCR
- ESTATUS_CASO
- dictamen.DESCRIPCIÓN
- estatus_caso.DESCRIPCIÓN 

donde el estado es Sonora, esto es, ENTIDAD_RES es igual a 26.

In [36]:
# Consulta SQL
query = """
    SELECT
        ID_REGISTRO,
        FECHA_ACTUALIZACION,
        EDAD_ANOS,
        RESULTADO_PCR,
        ESTATUS_CASO,
        dictamen.DESCRIPCIÓN AS DICTAMEN_DESC,
        municipio.MUNICIPIO AS MUNICIPIO_DESC,
        sexo.DESCRIPCIÓN AS SEXO_DESC
    FROM
        registro
        INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
        INNER JOIN sexo ON registro.SEXO = sexo.CLAVE
        INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
    WHERE
        ENTIDAD_RES = 26
        AND ESTATUS_CASO = 2
        AND DICTAMEN IN (1, 4, 5);
"""

# Lectura de datos desde SQLite
df = pd.read_sql_query(query, connection)

# Imprimir el DataFrame resultante
df.head()

,ID_REGISTRO,FECHA_ACTUALIZACION,EDAD_ANOS,RESULTADO_PCR,ESTATUS_CASO,DICTAMEN_DESC,MUNICIPIO_DESC,SEXO_DESC
0,910978,2022-01-06,39,2,2,NO APLICA,BÁCUM,MUJER
1,911682,2022-01-06,12,5,2,NO APLICA,BÁCUM,MUJER
2,926816,2022-01-06,29,5,2,NO APLICA,NAVOJOA,MUJER
3,925132,2022-01-06,28,5,2,NO APLICA,CAJEME,MUJER
4,923452,2022-01-06,13,3,2,NO APLICA,CAJEME,HOMBRE


In [37]:
sexo = df["SEXO_DESC"].value_counts()

# Crear un DataFrame para el gráfico
df_total_sexo = sexo.reset_index()
df_total_sexo.columns = ['SEXO_DESC', 'Total_Casos']
df_total_sexo

,SEXO_DESC,Total_Casos
0,MUJER,12884
1,HOMBRE,10129


In [38]:
# Crear gráfico de barras
fig = px.bar(df_total_sexo, x='SEXO_DESC', y="Total_Casos", color='SEXO_DESC',
             title='Diferencia entre Hombres y mujeres en total de Casos',
             labels={'count': 'Total de casos', 'SEXO_DESC': 'Género', "Total_Casos" : "Total de casos"},
             color_discrete_sequence=['purple', 'teal'])

# Mostrar el gráfico interactivo
fig.show()

Este hallazgo respalda los resultados obtenidos a nivel federal, donde se observa una prevalencia mayor de casos de dengue en mujeres. Este patrón puede deberse a diversos factores, como diferencias biológicas en la respuesta inmunológica, comportamientos relacionados con la exposición al mosquito transmisor o incluso sesgos en la notificación de casos. 

In [39]:
# Agrupar por edad y sexo y contar el número de ocurrencias en cada grupo
conteo_por_grupo = df.groupby(['EDAD_ANOS', 'SEXO_DESC']).size().reset_index(name='conteo')
conteo_por_grupo = conteo_por_grupo.pivot(index='EDAD_ANOS', columns='SEXO_DESC', values='conteo').reset_index()
conteo_por_grupo


SEXO_DESC,EDAD_ANOS,HOMBRE,MUJER
0,0,72.0,72.0
1,1,4.0,15.0
2,2,52.0,27.0
3,3,25.0,22.0
4,4,42.0,57.0
...,...,...,...
85,85,45.0,5.0
86,87,13.0,2.0
87,89,14.0,NaN
88,90,NaN,23.0


In [40]:
# Definir los bins de edades
bins = [0,10, 20, 30, 40, 50, 60, 70, 80, 90]

# Crear la columna de intervalos de edades
conteo_por_grupo['EDAD_ANOS'] = pd.cut(conteo_por_grupo['EDAD_ANOS'], bins=bins, labels=['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89'], right=False)
conteo_por_grupo = conteo_por_grupo.groupby("EDAD_ANOS").agg(sum)
conteo_por_grupo = conteo_por_grupo.reset_index(drop=False)
conteo_por_grupo

SEXO_DESC,EDAD_ANOS,HOMBRE,MUJER
0,0-9,679.0,771.0
1,10-19,2398.0,2313.0
2,20-29,1958.0,2736.0
3,30-39,1488.0,2253.0
4,40-49,1490.0,1932.0
5,50-59,965.0,1367.0
6,60-69,673.0,958.0
7,70-79,332.0,409.0
8,80-89,146.0,111.0


In [41]:

# Crear gráfica de barras apiladas
fig = px.bar(conteo_por_grupo, x='EDAD_ANOS', y=['MUJER','HOMBRE'], barmode='group',
             labels={'value': 'Cantidad', 'variable': 'Sexo', 'EDAD_ANOS': 'Rango de edad'},
             title='Distribución de edades por sexo de casos de dengue',
             color_discrete_sequence=['purple', 'teal'])

# Mostrar el gráfico interactivo
fig.show()

Esta visualización nos revela dos tendencias notables:
Se observa una prevalencia más alta de mujeres con dengue en todos los grupos de edad, con una única excepción.
Ambas distribuciones muestran similitudes, exhibiendo un sesgo hacia la izquierda. Además, es interesante destacar que tienen modas diferentes.
Este fenómeno podría estar vinculado a diversos factores, como comportamientos específicos de género, respuestas inmunológicas diferenciadas o incluso variaciones en la exposición al mosquito vector en distintos grupos de edad.

In [42]:
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23013 entries, 0 to 23012
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_REGISTRO          23013 non-null  int64         
 1   FECHA_ACTUALIZACION  23013 non-null  datetime64[ns]
 2   EDAD_ANOS            23013 non-null  int64         
 3   RESULTADO_PCR        23013 non-null  int64         
 4   ESTATUS_CASO         23013 non-null  int64         
 5   DICTAMEN_DESC        23013 non-null  object        
 6   MUNICIPIO_DESC       23013 non-null  object        
 7   SEXO_DESC            23013 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 1.4+ MB


### 

In [43]:
query_serotipo = """
    SELECT
        PCR_DESC,
        COUNT(*) AS TOTAL_CASOS
        
    FROM
        (
            SELECT
                resultado_pcr.DESCRIPCIÓN as PCR_DESC
            FROM
                registro
                INNER JOIN resultado_pcr ON registro.RESULTADO_PCR = resultado_pcr.CLAVE
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_CASO = 2
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        PCR_DESC;
"""

# Lectura de datos desde SQLite
df = pd.read_sql_query(query_serotipo, connection)

# Imprimir el DataFrame resultante
df = df.sort_values("TOTAL_CASOS", ascending=False)
df

,PCR_DESC,TOTAL_CASOS
3,SIN SEROTIPO AISLADO,16460
1,DENV2,6056
2,DENV3,463
0,DENV1,34


In [44]:
# Crear gráfico de barras
fig = px.bar(
    df,
    x='PCR_DESC',
    y='TOTAL_CASOS',
    color='PCR_DESC',
    title='Diferencia entre los diferentes serotipos de dengue confirmados en el estado',
    labels={'Total_Casos': 'Total de casos', 'PCR_DESC': 'Serotipo del virus del dengue '},
    color_discrete_sequence=['purple', 'teal', 'green', 'orange']  # Specify four distinct colors
)

# Mostrar el gráfico interactivo
fig.show()


En la visualización anterior podemos notar que el tipo de dengue que más predomina en el estado es "Sin serotipo aislado".

In [45]:
query_pcr_hist = """
    SELECT
        PCR_DESC,
        FECHA_SIGN_SINTOMAS,
        COUNT(*) AS TOTAL_CASOS
    FROM
        (
            SELECT
                FECHA_SIGN_SINTOMAS,
                resultado_pcr.DESCRIPCIÓN as PCR_DESC
            FROM
                registro
                INNER JOIN resultado_pcr ON registro.RESULTADO_PCR = resultado_pcr.clave
                INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_CASO = 2
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        FECHA_SIGN_SINTOMAS, PCR_DESC;
"""

# Lectura de datos desde SQLite
df_pcr_hist = pd.read_sql_query(query_pcr_hist, connection)

# Imprimir el DataFrame resultante
df_pcr_hist

,PCR_DESC,FECHA_SIGN_SINTOMAS,TOTAL_CASOS
0,SIN SEROTIPO AISLADO,01/06/2022,4
1,DENV2,01/07/2022,2
2,SIN SEROTIPO AISLADO,02/06/2022,4
3,DENV2,02/07/2022,1
4,SIN SEROTIPO AISLADO,03/06/2022,3
...,...,...,...
429,SIN SEROTIPO AISLADO,29/05/2022,1
430,DENV2,29/06/2022,1
431,DENV2,30/05/2022,1
432,SIN SEROTIPO AISLADO,30/05/2022,2


In [46]:
#Se cambian a objetos datetime los formatos mixtos
date1 = pd.to_datetime(df_pcr_hist['FECHA_SIGN_SINTOMAS'], errors='coerce', format='%Y-%m-%d')
date2 = pd.to_datetime(df_pcr_hist['FECHA_SIGN_SINTOMAS'], errors='coerce', format='%d/%m/%Y')
df_pcr_hist['FECHA_SIGN_SINTOMAS'] = date1.fillna(date2)

#Se organizan las fechas
df_pcr_hist = df_pcr_hist.sort_values(by='FECHA_SIGN_SINTOMAS')

# Crear una nueva columna 'MES_ANO' que contenga la combinación de mes y año
df_pcr_hist['MES_ANO'] = df_pcr_hist['FECHA_SIGN_SINTOMAS'].dt.to_period('M')

# Convertir 'MES_ANO' a string
df_pcr_hist['MES_ANO'] = df_pcr_hist['MES_ANO'].astype(str)

# Agrupar por 'MES_ANO' y 'PCR_DESC', sumando los casos
df_pcr_hist = df_pcr_hist.groupby(['MES_ANO', 'PCR_DESC']).agg({'TOTAL_CASOS': 'sum'}).reset_index()

df_pcr_hist.head()

,MES_ANO,PCR_DESC,TOTAL_CASOS
0,2021-02,SIN SEROTIPO AISLADO,1
1,2021-09,DENV2,6
2,2021-09,SIN SEROTIPO AISLADO,37
3,2021-10,DENV2,3
4,2021-10,SIN SEROTIPO AISLADO,28


In [47]:
# Crea la gráfica interactiva con Plotly Express
fig = px.line(df_pcr_hist, x='MES_ANO', y='TOTAL_CASOS', color='PCR_DESC',
              labels={'TOTAL_CASOS': 'Total de Casos', 'FECHA_ACTUALIZACION': 'Fecha de Actualización'},
              title='Comportamiento Histórico de casos totales por tipo de serotipo')

# Muestra la gráfica interactiva
fig.show()

En la representación gráfica, se destaca claramente un aumento significativo en los casos de dengue durante el mes de septiembre. Este incremento podría estar estrechamente vinculado a variables climáticas, especialmente considerando la estacionalidad del dengue y su relación con las condiciones ambientales propicias para la proliferación del mosquito vector.

Adicionalmente, es interesante notar que el serotipo "Denv2" ocupa la segunda posición en cuanto a incidencia. 

In [48]:
query_hist = """
    SELECT
        FECHA_SIGN_SINTOMAS,
        COUNT(*) AS TOTAL_CASOS
    FROM
        (
            SELECT
                FECHA_SIGN_SINTOMAS
            FROM
                registro
                INNER JOIN municipio ON registro.MUNICIPIO_RES = municipio.CLAVE_MUNICIPIO AND registro.ENTIDAD_RES = municipio.CLAVE_ENTIDAD
            WHERE
                ENTIDAD_RES = 26
                AND ESTATUS_CASO = 2
                AND DICTAMEN IN (1, 4, 5)
        ) AS filtered_data
    GROUP BY
        FECHA_SIGN_SINTOMAS;
"""

# Lectura de datos desde SQLite
df_pcr_hist = pd.read_sql_query(query_hist, connection)

# Imprimir el DataFrame resultante
df_pcr_hist.head()

,FECHA_SIGN_SINTOMAS,TOTAL_CASOS
0,01/06/2022,4
1,01/07/2022,2
2,02/06/2022,4
3,02/07/2022,1
4,03/06/2022,3


In [49]:
#Se cambian a objetos datetime los formatos mixtos
date1 = pd.to_datetime(df_pcr_hist['FECHA_SIGN_SINTOMAS'], errors='coerce', format='%Y-%m-%d')
date2 = pd.to_datetime(df_pcr_hist['FECHA_SIGN_SINTOMAS'], errors='coerce', format='%d/%m/%Y')
df_pcr_hist['FECHA_SIGN_SINTOMAS'] = date1.fillna(date2)

#Se organizan las fechas
df_pcr_hist = df_pcr_hist.sort_values(by='FECHA_SIGN_SINTOMAS')
df_pcr_hist


,FECHA_SIGN_SINTOMAS,TOTAL_CASOS
25,2021-02-08,1
26,2021-09-09,1
27,2021-09-15,1
28,2021-09-18,1
29,2021-09-19,2
...,...,...
245,2022-11-27,2
246,2022-11-28,4
247,2022-11-29,2
248,2022-12-01,2


In [50]:
# Crea la gráfica interactiva con Plotly Express
fig = px.line(df_pcr_hist, x='FECHA_SIGN_SINTOMAS', y='TOTAL_CASOS',
              labels={'TOTAL_CASOS': 'Total de Casos', 'FECHA_ACTUALIZACION': 'Fecha de Actualización', 'FECHA_SIGN_SINTOMAS': 'Fecha de inicio de síntomas del dengue'},
              title='Comportamiento Histórico del total de casos por tipo de serotipo')

# Muestra la gráfica interactiva
fig.show()

La gráfica revela de manera destacada dos picos de casos de dengue, ambos situados en el mes de septiembre, con un máximo que supera los 500 casos confirmados. Estos picos están claramente correlacionados con las temporadas de lluvia. Esta asociación entre la incidencia del dengue y las lluvias es un patrón bien conocido debido a que las precipitaciones crean condiciones ideales para la reproducción del mosquito Aedes, el vector responsable de la transmisión del virus del dengue.

In [51]:
connection.close()